In [1]:
from vllm import LLM, SamplingParams
import torch
import numpy as np

In [2]:
from lmlogits import TokTrie, RegexRecognizer

In [3]:
llm = LLM(model="TheBloke/Mistral-7B-Instruct-v0.2-AWQ", quantization="awq", max_model_len=4096, gpu_memory_utilization=0.8)

WARNING 04-20 02:04:12 config.py:164] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-20 02:04:12 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)
INFO 04-20 02:04:13 utils.py:580] Found nccl from library /home/lucas/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 04-20 02:04:14 selector.py:65] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-20 02:04:14 selector.py:33] Using X

In [4]:
trie = TokTrie('/home/lucas/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-AWQ/snapshots/f970a2bb89d5c2f9d217dc337f39e24625d6462a/tokenizer.json')

In [5]:
BULLET_LIST_REGEX = "(- [^\n]*\n)+(- [^\n]*)(\n\n)?"

In [7]:
class BaseLogitsProcessor:

    def __init__(self, rx_string: str):
        self.rx = RegexRecognizer(rx_string)

    def __call__(self, input_ids,
                 scores):

        if len(input_ids) != 0:
            last_token = input_ids[-1]
            self.rx.append_token(trie, last_token)
        
        allowed_tokens = self.rx.compute_bias(trie)
        scores.add_(torch.FloatTensor(allowed_tokens).to(scores.device))
        return scores

In [13]:
prompts = [
    "The most funny jokes are:\n\n"
    # "Hello, my name is",
    # "The president of the United States is",
    # "The capital of France is",
    # "The future of AI is",
]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, logits_processors=[BaseLogitsProcessor(BULLET_LIST_REGEX)], max_tokens=128)
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text)
    # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it]

- Why don't scientists trust atoms? Because they make up everything.
- Why did the chicken go to the seance? To talk to the other side.
- What do you call a fake noodle? An impasta.
- Why did the scarecrow win an award? Because he was outstanding in his field.
- What do you call a bear with no teeth? A gummy bear.
- Why did the bicycle fall over? Because it was two-tired.
- Why did the tomato turn red? Because it saw the salad dressing.
- Why did the
